In [1]:
import pandas as pd
import numpy as np
import torch
#!pip install fastai==0.7.0
#!pip install torch
import fastai
from fastai.imports import *
from fastai.torch_imports import *
from fastai.core import *
from fastai.model import fit
from fastai.dataset import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *
from fastai.text import *

import dill as pickle
#!pip install spacy
#!python -m spacy download en
import spacy
import html

import pickle
import collections
#!pip install nbimporter
import nbimporter

In [2]:
#Das kommt ganz am Anfang, mit kleinem Text dass wir um den Code schlank zu halten einige Teile in Funktionen auslagern
import functions as fct

Importing Jupyter notebook from functions.ipynb


In [3]:
with open('wt103/itos_wt103.pkl', 'rb') as pickle_file:
    itos_wiki = pickle.load(pickle_file)
    
stoi_wiki = collections.defaultdict(lambda:-1, {v:k for k,v 
                                              in enumerate(itos_wiki)})

In [4]:
from functions import Prep

sent = 'Yesterday I flew with a new airline and it was really'
tok_sent = Prep.tokenize(sent, stoi_wiki)
print(tok_sent)

0
{'Tokens': ['\n', 'xbos', 'yesterday', 'i', 'flew', 'with', 'a', 'new', 'airline', 'and', 'it', 'was', 'really'], 'Encoded_Tokens': [288, 303, 13256, 74, 3273, 19, 9, 58, 4745, 6, 29, 12, 1041]}


In [5]:
onehot = Prep.OneHot(tok_sent['Encoded_Tokens'], dimension = len(itos_wiki))
print(onehot)
onehot.shape

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


(13, 238462)

In [6]:
lm_wgts_wiki = torch.load('wt103/fwd_wt103.h5', map_location=lambda storage, 
                  loc: storage)

embedding = np.matmul(onehot, lm_wgts_wiki['0.encoder.weight'])
print(embedding)
embedding.shape

[[-0.4396  -0.39229 -0.12621 ...  0.12117  0.2433   0.10483]
 [-0.16437 -0.35955 -0.04734 ...  0.04385  1.08976 -0.09999]
 [ 0.28111 -0.59665  0.1551  ... -0.10522  0.31061  0.24847]
 ...
 [-0.10496  0.75285 -0.14176 ...  0.06853  0.02307  0.0305 ]
 [ 0.03717  0.86379 -0.01512 ... -0.01448 -0.64909 -0.00839]
 [ 0.6857  -0.79381 -0.18487 ... -0.30761  0.44434 -0.31734]]


(13, 400)

In [7]:
from functions import LSTM
lstm  = LSTM(embedding, lm_wgts_wiki, 1150)
st_lstm = LSTM.stacked(lstm)
print(st_lstm.hidden_state_l2)
st_lstm.hidden_state_l2.shape

[[-0.03709  0.19475 -0.01835 ... -0.00458  0.13194  0.01884]
 [-0.20202  0.13205 -0.01257 ... -0.01585  0.15082  0.10082]
 [-0.28976  0.1191  -0.10623 ... -0.10468 -0.30108  0.51579]
 ...
 [ 0.05157  0.11202  0.00307 ... -0.39102 -0.0455  -0.00869]
 [-0.0446   0.05912 -0.02197 ... -0.37411  0.04055 -0.33509]
 [-0.0128   0.10315 -0.11878 ... -0.39192 -0.02202 -0.03815]]


(13, 400)

In [8]:
from functions import Prediction
prediction = Prediction.predict(st_lstm.hidden_state_l2, lm_wgts_wiki, stoi_wiki, tok_sent, 10)
print(prediction[0], "\n", prediction[1])

['\n xbos yesterday i flew with a new airline and it was really'] 
 [           word  probability
9             a     0.000013
2           the     0.000010
42          not     0.000009
3             ,     0.000005
8            to     0.000004
415        good     0.000004
1077     likely     0.000004
587   important     0.000004
284       great     0.000003
127        well     0.000003]


In [9]:
# loading class Classifier from functions notebook
from functions import Classifier

# loading the twitter dataset vocabulary
with open('lm/itos.pkl', 'rb') as pickle_file:
    itos = pickle.load(pickle_file)

stoi = collections.defaultdict(lambda:0, 
                               {v:k for k,v in enumerate(itos)})

# loading the training model parameters of the classifier
clas_wgts = torch.load('models/clas_2.h5', map_location=lambda storage, 
                  loc: storage)

# prep functions executes tokenization, one-hot encoding, embedding
# and lstm forward pass, outputs hidden state tensor of last lstm
hidden_state = Classifier.prep('Yesterday I flew with a new airline and it was well', clas_wgts, stoi)
print(hidden_state)
hidden_state.shape

0
[[-0.00927  0.05593 -0.01051 ... -0.00032  0.09934  0.00069]
 [-0.02445  0.08898 -0.05619 ...  0.0116   0.05344  0.03297]
 [ 0.01426  0.07458 -0.00367 ... -0.01137 -0.09863  0.12703]
 ...
 [-0.09402  0.06873 -0.01269 ... -0.09954 -0.07107  0.02303]
 [-0.05729  0.06447 -0.05503 ... -0.18477  0.00308 -0.1477 ]
 [ 0.00681  0.03628 -0.2629  ... -0.16824 -0.09854  0.03765]]


(13, 400)

In [10]:
# apply concat pooling for last hidden state vector (representing last word of tweet)
concat = Classifier.concat_pooling(hidden_state)
print(concat)
concat.shape

[ 0.00681  0.03628 -0.2629  ... -0.12163 -0.03254  0.00786]


(1200,)

In [11]:
# process concat pooled hidden state in relu layer
relu = Classifier.relu_layer(concat, clas_wgts)
print(relu)
relu.shape

[[0.      0.      1.38399 2.61893 0.01491 1.84187 0.      0.      1.0487  0.      0.      1.29328 0.
  0.      3.78879 2.94374 0.9839  0.      0.      1.80533 2.62585 0.      1.06425 0.      0.94657 0.4659
  0.      0.12778 0.      0.      0.      1.04623 0.      0.      0.      2.42056 0.      0.      1.62496
  0.      2.32776 0.      2.13438 0.44674 0.      1.20542 2.44793 1.09395 0.      0.     ]]


(1, 50)

In [12]:
# process output from relu layer in softmax layer and print out prediction
prediction = Classifier.clas_predict(relu, clas_wgts)
print(prediction)

{'negative': 0.002050045832835973, 'neutral': 0.022765535247444953, 'positive': 0.975184418919719}
